# NAFION IONOMER
## `DREIDING` PARAMETRIZATION AND `ESP method of Merz-Kollman` CHARGES

In [1]:
! pwd

/home/kchinas/Documents/PhD/LAMMPS/CL/2ndMODEL/3rdAttmp/ionomer


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='ticks')

import re
import os

def get_file(filename):
    """
    returns raw dataframe from any type of file
    """
    with open(filename, "r") as file:
        file_content = file.readlines()
    df = pd.DataFrame(file_content, columns=['data'])
    return df

In [3]:
! ls

DREIDING.pdf			ionomer_DREIDING.lammps
generate_mol_file.ipynb		O5R9_allatom_optimised_geometry.pdb
ionomer_DREIDING_correct_Q.mol	O5R9_allatom_optimized_geometry.lt


In [4]:
raw_ionomer_ATB   = get_file("O5R9_allatom_optimized_geometry.lt")
raw_ionomer_MATSQ = get_file("ionomer_DREIDING.lammps")

In [5]:
def get_ATB_data(ionomer):    
    data = ionomer[ionomer["data"].str.contains(r"\$atom\:[A-Z]\d+\s+\$mol")].reset_index(drop=True)
    data["data"] = data["data"].str.split('\s')
    data["data"] = data["data"].apply(lambda x: list(filter(None, x)))
    data["atom"] = data["data"].apply(lambda x: x[0].replace("$atom:", ""))
    data["elements"] = data["atom"].str.replace(r"\d+", "", regex=True)
    data["gromos elemts"] = data["data"].apply(lambda x: x[2].replace("@atom:", ""))
    data["charge"] = data["data"].apply(lambda x: x[3]).astype(float)
    data["x"] = data["data"].apply(lambda x: x[4]).astype(float)
    data["y"] = data["data"].apply(lambda x: x[5]).astype(float)
    data["z"] = data["data"].apply(lambda x: x[6]).astype(float)
    data.drop("data", axis=1, inplace=True)
    return data

def get_ATB_bonds(ionomer):
    data = ionomer[ionomer["data"].str.contains(r"\$bond\:b")].reset_index(drop=True)
    data["data"] = data["data"].str.split('\s')
    data["data"] = data["data"].apply(lambda x: list(filter(None, x)))
    data["b"] = data["data"].apply(lambda x: x[0].replace("$bond:b", "")) # index
    data["g"] = data["data"].apply(lambda x: x[1].replace("@bond:g", "")) # type
    data["atom1"] = data["data"].apply(lambda x: x[2].replace("$atom:", ""))
    data["atom2"] = data["data"].apply(lambda x: x[3].replace("$atom:", ""))
    data.drop("data", axis=1, inplace=True)
    return data

def get_MATSQ_data(ionomer,slices):
    data = ionomer.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    data["data"] = data["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    data["atom id"] = data["data"].apply(lambda x: x[0]).astype(int)
    data["molecule id"] = data["data"].apply(lambda x: x[1]).astype(int)
    data["atom type"] = data["data"].apply(lambda x: x[2]).astype(int)
    data["charge"] = data["data"].apply(lambda x: x[3]).astype(float)
    data["x"] = data["data"].apply(lambda x: x[4]).astype(float)
    data["y"] = data["data"].apply(lambda x: x[5]).astype(float)
    data["z"] = data["data"].apply(lambda x: x[6]).astype(float)
    data["dreiding elemts"] = data["atom type"].map({1:"F_",2:"C_3",3:"O_3",4:"O_2",5:"S_3"})
    return data

def get_MATSQ_bonds(ionomer,slices):
    data = ionomer.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    data["data"] = data["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    data["bond id"] = data["data"].apply(lambda x: x[0]).astype(int)
    data["bond type"] = data["data"].apply(lambda x: x[1]).astype(int)
    data["atom1"] = data["data"].apply(lambda x: x[2]).astype(int)
    data["atom2"] = data["data"].apply(lambda x: x[3]).astype(int)
    return data

ATB_structure = get_ATB_data(raw_ionomer_ATB)
MATSQ_structure = get_MATSQ_data(raw_ionomer_MATSQ,[27,524])

MATSQ_bonds = get_MATSQ_bonds(raw_ionomer_MATSQ,[528,1024])
ATB_bonds   = get_ATB_bonds(raw_ionomer_ATB)

##  [C$_{19}$F$_{39}$O$_5$S$^-$](https://atb.uq.edu.au/molecule.py?molid=1723660) AND C$_{152}$F$_{298}$O$_{40}$S$_8^-$ CHARGE COMPARISON 

In [6]:
# ESP method of Merz-Kollman
ATB_structure["charge"].sum()

-0.9999999999999993

In [7]:
#ATB_structure["charge"].unique() # 48 unique charges

In [8]:
# Qeq (Chargue equilibration)
MATSQ_structure["charge"].sum()

-0.000469999999992865

In [9]:
#MATSQ_structure["charge"].unique() # 40 unique charges

### TASK: fix ionomer charges

##### ATB Charge Distribution

* [Ionomer](https://atb.uq.edu.au/viewer.py?molid=1723660)
* [Polymer](https://atb.uq.edu.au/viewer.py?molid=1705448)

In [10]:
ATB_structure["gromos elemts"].unique() 

array(['F', 'CPos', 'OE', 'C', 'SDmso', 'OM'], dtype=object)

In [11]:
ATB_structure[ATB_structure["gromos elemts"].isin(["OM"])].charge.unique() # This is Dreiding O_2

array([-1.019])

In [12]:
ATB_structure[ATB_structure["gromos elemts"].isin(["OE"])].charge.unique() # This is Dreiding O_3
# -0.301 is closer to S
# -0.292 is closer to main chain

array([-0.292, -0.301])

In [13]:
ATB_structure[ATB_structure["gromos elemts"].isin(["F"])].charge.unique() # C - (F)

array([-0.132, -0.11 , -0.107, -0.108, -0.106, -0.104, -0.111, -0.113,
       -0.117, -0.12 , -0.099, -0.075, -0.141, -0.084, -0.072, -0.09 ,
       -0.125, -0.102, -0.142, -0.116, -0.152, -0.092, -0.143, -0.15 ,
       -0.164, -0.17 , -0.159])

In [14]:
ATB_structure[ATB_structure["gromos elemts"].isin(["CPos"])].charge.unique() # (C) - F

array([0.395, 0.203, 0.209, 0.21 , 0.212, 0.206, 0.23 , 0.234, 0.276,
       0.216, 0.189, 0.384, 0.348, 0.256, 0.373, 0.339])

In [15]:
ATB_structure[ATB_structure["gromos elemts"].isin(["C"])]#.charge.unique() # (C) - S

,atom,elements,gromos elemts,charge,x,y,z
57,C19,C,C,-0.212,5.169005,2.481924,-0.463252


In [16]:
ATB_structure[ATB_structure["gromos elemts"].isin(["SDmso"])].charge.unique() # S

array([2.878])

##### MATSQ Charge Distribution

In [17]:
MATSQ_structure[MATSQ_structure["dreiding elemts"].isin(["F_"])].charge.unique() # C - F

array([-0.19227, -0.16086, -0.19226, -0.18904, -0.19211, -0.19225,
       -0.1925 , -0.19264, -0.19498, -0.19816, -0.16126, -0.16734,
       -0.19218, -0.16112, -0.16583, -0.17404])

In [18]:
MATSQ_structure[MATSQ_structure["dreiding elemts"].isin(["C_3"])].charge.unique()

array([0.38416, 0.4499 , 0.3845 , 0.40751, 0.38451, 0.38733, 0.38465,
       0.3843 , 0.38243, 0.37963, 0.37461, 0.3515 , 0.44502, 0.41153,
       0.41137, 0.39254, 0.44746, 0.42584, 0.4685 ])

#### For now, I will do a quick dirty fix for the side chains, but later I could try to fix everything based on bonds.

In [19]:
id2elemt = dict(zip(MATSQ_structure["atom id"].values, MATSQ_structure["dreiding elemts"].values))

MATSQ_bonds["elemnt1"] = MATSQ_bonds["atom1"].map(id2elemt)
MATSQ_bonds["elemnt2"] = MATSQ_bonds["atom2"].map(id2elemt)
MATSQ_bonds["flag charge"] = [0]*len(MATSQ_bonds)

### First, changing those associated to Sulfur

In [20]:
QckFixRes = (-8+7.4851)/72

In [21]:
MATSQ_bonds.loc[MATSQ_bonds[MATSQ_bonds["elemnt2"].isin(["S_3"])].index,"flag charge"] = 1

In [22]:
update_id_list = pd.Series(list(MATSQ_bonds[MATSQ_bonds["flag charge"].isin([1])]["atom1"].values) + 
                           list(MATSQ_bonds[MATSQ_bonds["flag charge"].isin([1])]["atom2"].values)).unique()

update = MATSQ_structure[MATSQ_structure["atom id"].isin(update_id_list)]

MATSQ_structure.loc[update[update["dreiding elemts"].isin(["C_3"])].index,"charge"] = \
                ATB_structure[ATB_structure["gromos elemts"].isin(["C"])].charge.unique()[0] + QckFixRes

MATSQ_structure.loc[update[update["dreiding elemts"].isin(["S_3"])].index,"charge"] = \
                ATB_structure[ATB_structure["gromos elemts"].isin(["SDmso"])].charge.unique()[0] + QckFixRes

MATSQ_structure.loc[update[update["dreiding elemts"].isin(["O_2"])].index,"charge"] = \
                ATB_structure[ATB_structure["gromos elemts"].isin(["OM"])].charge.unique()[0] + QckFixRes

### Second, changing those associated to Oxygen close to the PTFE backbone

In [23]:
MATSQ_bonds.loc[MATSQ_bonds[MATSQ_bonds["elemnt2"].isin(["O_3"])].index,"flag charge"] = 2

#### Next task: discern which O is closer to S and which one is closer to the PTFE backbone. 

For this, I am also going to change the other Carbon charge that is in between S and O. 

In [24]:
update = MATSQ_bonds[MATSQ_bonds["flag charge"].isin([1]) & MATSQ_bonds["elemnt1"].isin(["C_3"])]
update_id_list = list(update["atom1"].values)

In [25]:
# All atoms bonded to the updated carbon is flag #3
MATSQ_bonds.loc[MATSQ_bonds[MATSQ_bonds["atom2"].isin(update_id_list) & 
                            MATSQ_bonds["elemnt1"].isin(["C_3"])].index, "flag charge"] = 3
# They should have C18 charge
second_C_list = list(MATSQ_bonds[MATSQ_bonds["flag charge"].isin([3])]["atom1"].values) 

MATSQ_structure.loc[MATSQ_structure[MATSQ_structure["atom id"].isin(second_C_list)].index,"charge"] = \
                ATB_structure[ATB_structure["atom"].isin(["C18"])]["charge"].values[0] + QckFixRes

Now, I will update the O charges based on which ones is closer to S and the PTFE backbone, respectively.

In [26]:
O_close_to_S = list(MATSQ_bonds[MATSQ_bonds["flag charge"].isin([2]) & 
                           MATSQ_bonds["atom1"].isin(second_C_list)]["atom2"].values)

MATSQ_structure.loc[MATSQ_structure[MATSQ_structure["atom id"].isin(O_close_to_S)].index,"charge"] = \
                ATB_structure[ATB_structure["gromos elemts"].isin(["OE"])].charge.unique()[0] + QckFixRes

In [27]:
O_close_to_PTFE = MATSQ_bonds[MATSQ_bonds["flag charge"].isin([2]) & 
                    ~MATSQ_bonds["atom2"].isin(O_close_to_S)]["atom2"].unique()

MATSQ_structure.loc[MATSQ_structure[MATSQ_structure["atom id"].isin(O_close_to_PTFE)].index,"charge"] = \
                ATB_structure[ATB_structure["gromos elemts"].isin(["OE"])].charge.unique()[1] + QckFixRes

In [28]:
# Now, the molecule has a net charge closer to the one expected
MATSQ_structure["charge"].sum()

-8.000058888888873

# REMARKS | [GROMOS 54A7 PARAMETRIZATION](https://atb.uq.edu.au/download.py?lt=54A7)

Bonding Characteristics

* S=O: g13: $k_{harmonic}$ ~ 459, $r_0$ ~ 1.36
* C-O(C): g15: $k_{harmonic}$ ~ 399, $r_0$ ~ 1.39
* S-O(H): g28: $k_{harmonic}$ ~ 299, $r_0$ ~ 1.61

### NEXT TASK: create mol file for LAMMPS usage.

Remember to add the double bonds.

In [29]:
def get_angles(data, slices):
    angles = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    angles["data"] = angles["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    angles["angle id"] = angles["data"].apply(lambda x: x[0]).astype(int)
    angles["angle type"] = angles["data"].apply(lambda x: x[1]).astype(int)
    angles["atom1"] = angles["data"].apply(lambda x: x[2]).astype(int)
    angles["atom2"] = angles["data"].apply(lambda x: x[3]).astype(int)
    angles["atom3"] = angles["data"].apply(lambda x: x[4]).astype(int)
    return angles


def get_dihedrals(data,slices):
    dihedrals = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    dihedrals["data"] = dihedrals["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    dihedrals["dihedral id"] = dihedrals["data"].apply(lambda x: x[0]).astype(int)
    dihedrals["dihedral type"] = dihedrals["data"].apply(lambda x: x[1]).astype(int)
    dihedrals["atom1"] = dihedrals["data"].apply(lambda x: x[2]).astype(int)
    dihedrals["atom2"] = dihedrals["data"].apply(lambda x: x[3]).astype(int)
    dihedrals["atom3"] = dihedrals["data"].apply(lambda x: x[4]).astype(int)
    dihedrals["atom4"] = dihedrals["data"].apply(lambda x: x[5]).astype(int)
    return dihedrals

MATSQ_angles = get_angles(raw_ionomer_MATSQ,[1028,2003])
MATSQ_dihedrals = get_dihedrals(raw_ionomer_MATSQ,[2007,3389])

# FIX TYPE OF ATOM O_2 --> O_3

In [30]:
import itertools

def generate_array(n):
    return list(itertools.islice(itertools.cycle([1, 0, 0]), n))

In [31]:
change_O_type = MATSQ_bonds[MATSQ_bonds["elemnt2"].isin(["S_3"]) &  
                            MATSQ_bonds["elemnt1"].isin(["O_2"])].sort_values("atom2").reset_index(drop=True)
change_O_type["flag O"] = generate_array(len(change_O_type))
change_O_type = list(change_O_type[change_O_type["flag O"].isin([1])]["atom1"].values)

In [32]:
MATSQ_structure

,data,atom id,molecule id,atom type,charge,x,y,z,dreiding elemts
0,"[1, 1, 1, -0.19227, 11.69900000, 28.64700000, ...",1,1,1,-0.192270,11.699,28.647,15.056,F_
1,"[2, 1, 1, -0.19227, 32.48100000, 20.14400000, ...",2,1,1,-0.192270,32.481,20.144,13.951,F_
2,"[3, 1, 1, -0.16086, 19.44200000, 21.03100000, ...",3,1,1,-0.160860,19.442,21.031,13.741,F_
3,"[4, 1, 1, -0.19227, 39.16000000, 26.67400000, ...",4,1,1,-0.192270,39.160,26.674,20.750,F_
4,"[5, 1, 1, -0.19227, 41.45500000, 15.24100000, ...",5,1,1,-0.192270,41.455,15.241,22.698,F_
...,...,...,...,...,...,...,...,...,...
493,"[494, 1, 5, 0.41982, 39.57800000, 15.18500000,...",494,1,5,2.870849,39.578,15.185,13.208,S_3
494,"[495, 1, 5, 0.41982, 40.11500000, 23.53200000,...",495,1,5,2.870849,40.115,23.532,17.122,S_3
495,"[496, 1, 5, 0.41982, 17.18100000, 38.33000000,...",496,1,5,2.870849,17.181,38.330,14.255,S_3
496,"[497, 1, 5, 0.41982, 31.40600000, 19.22000000,...",497,1,5,2.870849,31.406,19.220,25.965,S_3


In [33]:
MATSQ_structure.loc[MATSQ_structure[MATSQ_structure["atom id"].isin(change_O_type)].index,
                    "atom type"] = 3

MATSQ_structure.loc[MATSQ_structure[MATSQ_structure["atom id"].isin(change_O_type)].index,
                    "dreiding elemts"] = "O_3"

### Add a new bonding style based on S-O_3, bond S=O_2 is already registered

In [34]:
MATSQ_bonds.loc[MATSQ_bonds[MATSQ_bonds["atom1"].isin(change_O_type)].index,"elemnt1"] = "O_3"
MATSQ_bonds.loc[MATSQ_bonds[MATSQ_bonds["atom1"].isin(change_O_type)].index,"bond type"] = 6

In [35]:
# Ions data
MATSQ_bonds[MATSQ_bonds["elemnt1"].isin(["O_3"])]

,data,bond id,bond type,atom1,atom2,elemnt1,elemnt2,flag charge
473,"[474, 5, 467, 491]",474,6,467,491,O_3,S_3,1
474,"[475, 5, 468, 492]",475,6,468,492,O_3,S_3,1
475,"[476, 5, 469, 493]",476,6,469,493,O_3,S_3,1
485,"[486, 5, 479, 495]",486,6,479,495,O_3,S_3,1
486,"[487, 5, 480, 496]",487,6,480,496,O_3,S_3,1
488,"[489, 5, 482, 497]",489,6,482,497,O_3,S_3,1
492,"[493, 5, 486, 494]",493,6,486,494,O_3,S_3,1
495,"[496, 5, 489, 498]",496,6,489,498,O_3,S_3,1


In [36]:
def write_mol_file():
    f = open("ionomer_DREIDING_correct_Q.mol", "w")
    f.write("# Created by Karina Chiñas Fuentes | Python | 29.08.24\n")
    f.write(f"\n{len(MATSQ_structure)} atoms\n{len(MATSQ_bonds)} bonds\n"+
            f"{len(MATSQ_angles)} angles\n{len(MATSQ_dihedrals)} dihedrals")
    f.write("\n\n")
    f.write("Coords\n\n")
    for i in range(len(MATSQ_structure)):
        f.write(f"{MATSQ_structure['atom id'][i]}\t{MATSQ_structure['x'][i]}"+
                f"\t{MATSQ_structure['y'][i]}\t{MATSQ_structure['z'][i]}\n")
    f.write("\nTypes\n\n")
    for i in range(len(MATSQ_structure)):
        f.write(f"{MATSQ_structure['atom id'][i]}\t{MATSQ_structure['atom type'][i]}\n")
    f.write("\nCharges\n\n")
    for i in range(len(MATSQ_structure)):
        f.write(f"{MATSQ_structure['atom id'][i]}\t{MATSQ_structure['charge'][i]}\n")
    f.write("\nBonds\n\n")
    for i in range(len(MATSQ_bonds)):
        f.write(f"{MATSQ_bonds['bond id'][i]}\t{MATSQ_bonds['bond type'][i]}\t"+
                f"{MATSQ_bonds['atom1'][i]}\t{MATSQ_bonds['atom2'][i]}\n")
    f.write("\nAngles\n\n")
    for i in range(len(MATSQ_angles)):
        f.write(f"{MATSQ_angles['angle id'][i]}\t{MATSQ_angles['angle type'][i]}\t"+
                f"{MATSQ_angles['atom1'][i]}\t{MATSQ_angles['atom2'][i]}\t{MATSQ_angles['atom3'][i]}\n")
    f.write("\nDihedrals\n\n")
    for i in range(len(MATSQ_dihedrals)):
        f.write(f"{MATSQ_dihedrals['dihedral id'][i]}\t{MATSQ_dihedrals['dihedral type'][i]}"+
                f"\t{MATSQ_dihedrals['atom1'][i]}\t{MATSQ_dihedrals['atom2'][i]}\t"+
                f"{MATSQ_dihedrals['atom3'][i]}\t{MATSQ_dihedrals['atom4'][i]}\n")

In [37]:
write_mol_file()

In [38]:
! code ionomer_DREIDING_correct_Q.mol